In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from sklearn.metrics import r2_score,max_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures,MinMaxScaler,StandardScaler
from statistics import mean
import datetime
from sklearn.svm import SVC
from scipy.optimize import curve_fit
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,confusion_matrix

In [79]:
train_data=pd.read_excel(r'C:\Users\taraf\Documents\CERTH_Train.xlsx')

In [80]:
Eval_DBS=pd.read_excel(r'C:\CERTH\EvaluationSet\DigitalBlurSet.xlsx')
Eval_NBS=pd.read_excel(r'C:\CERTH\EvaluationSet\NaturalBlurSet.xlsx')

In [63]:
test=pd.read_excel(r'C:\CERTH\EvaluationSet\test.xlsx')
testeval=pd.read_excel(r'C:\CERTH\EvaluationSet\testeval.xlsx')

In [17]:
testeval.tail()

,Image Name,Blur Label
1475,Nat_Original_996,-1
1476,Nat_Original_997,-1
1477,Nat_Original_998,-1
1478,Nat_Original_999,-1
1479,Nat_Original_1000,1


In [72]:
testevaltarger=testeval.iloc[:,-1]

In [73]:
testevaltarger

0       1
1       1
2       1
3       1
4       1
       ..
1475   -1
1476   -1
1477   -1
1478   -1
1479    1
Name: Blur Label, Length: 1480, dtype: int64

In [65]:
test=test.iloc[:,1:-1]

In [74]:
test.tail(10)

,Variance,Amax
1470,0.012307,0.925490
1471,0.005064,1.694118
1472,0.006496,0.898039
1473,0.014120,1.376471
1474,0.006211,1.113725
1475,0.013252,1.419608
1476,0.017624,1.482353
1477,0.004723,1.219608
1478,0.001966,1.243137
1479,0.009803,0.678431


In [81]:
Eval_DBS.head()

,MyDigital Blur,State
0,DiskR10_1.jpg,1
1,DiskR10_10.jpg,1
2,DiskR10_11.jpg,1
3,DiskR10_12.jpg,1
4,DiskR10_13.jpg,1


In [50]:
train_data.head()

,Variance,Amax
0,0.005518,1.082353
1,0.015117,1.176471
2,0.001532,0.898039
3,0.002433,0.376471
4,0.003662,0.803922


**Tain Data**

In [48]:
train_target=train_data.iloc[:,-1]

In [94]:
train_data=train_data.iloc[:,1:-2]

In [95]:
train_data.head(10)

,Variance,Amax
0,0.005518,1.082353
1,0.015117,1.176471
2,0.001532,0.898039
3,0.002433,0.376471
4,0.003662,0.803922
5,0.003545,0.941176
6,0.002834,0.349020
7,0.002756,0.341176
8,0.003058,0.294118
9,0.006061,0.419608


**Digital Blur Set Prediction**

In [82]:
Test_DBS=pd.read_excel(r'C:\Users\taraf\Documents\TestDigital.xlsx')

In [83]:
Test_DBS=Test_DBS.iloc[:,1:-1]

In [84]:
Test_DBS_Target=Eval_DBS.iloc[:,-1]

In [85]:
Test_DBS.head()

,Variance,Amax
0,0.001284,0.305882
1,0.007767,0.486275
2,0.001292,0.345098
3,0.002988,0.521569
4,0.006441,0.658824


In [52]:
params={
    "n_estimators":[10,20,50,100,500,700],
    "max_features":["auto","None","sqrt","log2",0.9,0.2],
    "min_samples_leaf":[1,2,3,4]
}

In [53]:
clf=RandomForestClassifier(random_state=42)

In [54]:
search=RandomizedSearchCV(clf,param_distributions=params,scoring='r2',n_jobs=-1,cv=5)

In [55]:
search.fit(train_data,train_target)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [479]:
search.best_params_

{'n_estimators': 500, 'min_samples_leaf': 3, 'max_features': 'log2'}

In [96]:
RandomModel=RandomForestClassifier(n_estimators=500,min_samples_leaf=5,max_features='log2',random_state=42)
modelRandom=RandomModel.fit(train_data,train_target)

In [90]:
DBS_RandomModel_pred=modelRandom.predict(Test_DBS)

In [87]:
confusion_matrix(testevaltarger,DBS_RandomModel_pred)

array([[516, 103],
       [298, 563]], dtype=int64)

In [104]:
print('Accuracy Score : ',accuracy_score(DBS_RandomModel_pred,Test_DBS_Target))
print('F1 Score : ',f1_score(DBS_RandomModel_pred,Test_DBS_Target))
print('Recall Score : ',recall_score(DBS_RandomModel_pred,Test_DBS_Target))
print('Precision Score : ',precision_score(DBS_RandomModel_pred,Test_DBS_Target))

Accuracy Score :  0.9229166666666667
F1 Score :  0.9573241061130334
Recall Score :  0.9952038369304557
Precision Score :  0.9222222222222223


**Natural Blur Set Prediction**

In [383]:
Test_NBS=pd.read_excel(r'C:\Users\taraf\Documents\TestNatural.xlsx')

In [385]:
Test_NBS=Test_NBS.iloc[:,1:-1]

In [386]:
Test_NBS_Target=Eval_NBS.iloc[:,-1]

In [387]:
Test_NBS.tail()

,Variance,Amax
995,0.013252,1.419608
996,0.017624,1.482353
997,0.004723,1.219608
998,0.001966,1.243137
999,0.009803,0.678431


In [556]:
params={
    "n_estimators":[10,20,50,100,500,700],
    "learning_rate":[0.05,0.5,0.7,1,1.2],
    "algorithm":['SAMME','SAMME.R']
 }

In [557]:
clf=AdaBoostClassifier(random_state=42)

In [558]:
search=RandomizedSearchCV(clf,param_distributions=params,scoring='r2',n_jobs=-1,cv=5)

In [559]:
search.fit(train_data,train_target)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                base_estimator=None,
                                                learning_rate=1.0,
                                                n_estimators=50,
                                                random_state=42),
                   iid='deprecated', n_iter=10, n_jobs=-1,
                   param_distributions={'algorithm': ['SAMME', 'SAMME.R'],
                                        'learning_rate': [0.05, 0.5, 0.7, 1,
                                                          1.2],
                                        'n_estimators': [10, 20, 50, 100, 500,
                                                         700]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='r2', verbose=0)

In [560]:
search.best_params_

{'n_estimators': 700, 'learning_rate': 1, 'algorithm': 'SAMME'}

In [561]:
AdaModel=AdaBoostClassifier(n_estimators=700,learning_rate=1,algorithm='SAMME')
modelAda=AdaModel.fit(train_data,train_target)

In [562]:
NBS_AdaModel_pred=modelAda.predict(Test_NBS)

In [563]:
accuracy_score(NBS_AdaModel_pred,Test_NBS_Target)

0.645

Random Forest

In [526]:
params={
    "n_estimators":[10,20,50,100,500,700],
    "max_features":["auto","None","sqrt","log2",0.9,0.2],
    "min_samples_leaf":[1,2,3,4]
}

In [527]:
clf=RandomForestClassifier(random_state=42)

In [528]:
search=RandomizedSearchCV(clf,param_distributions=params,scoring='r2',n_jobs=-1,cv=5)

In [529]:
search.fit(train_data,train_target)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [530]:
search.best_params_

{'n_estimators': 500, 'min_samples_leaf': 4, 'max_features': 'log2'}

In [550]:
RandomModel=RandomForestClassifier(n_estimators=500,min_samples_leaf=2,max_features='log2')
modelRandom=RandomModel.fit(train_data,train_target)

In [551]:
NBS_RandomModel_pred=modelRandom.predict(Test_NBS)

In [564]:
accuracy_score(NBS_RandomModel_pred,Test_NBS_Target)

0.645

**Model on NaturalBlur and DigitalBlur combined**

In [97]:
params={
    "n_estimators":[10,20,50,100,500,700],
    "max_features":["auto","None","sqrt","log2",0.9,0.2],
    "min_samples_leaf":[1,2,3,4]
}

In [98]:
clf=RandomForestClassifier(random_state=42)

In [99]:
search=RandomizedSearchCV(clf,param_distributions=params,scoring='r2',n_jobs=-1,cv=5)

In [126]:
search.fit(train_data,train_target)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [127]:
search.best_params_

{'n_estimators': 500, 'min_samples_leaf': 4, 'max_features': 'sqrt'}

In [128]:
RandomModel=RandomForestClassifier(n_estimators=500,min_samples_leaf=4,max_features='auto',random_state=42)
modelRandom=RandomModel.fit(train_data,train_target)

In [129]:
Test_RandomModel_pred=modelRandom.predict(test)

In [130]:
print('Accuracy Score : ',accuracy_score(Test_RandomModel_pred,testevaltarger))
print('F1 Score : ',f1_score(Test_RandomModel_pred,testevaltarger))
print('Recall Score : ',recall_score(Test_RandomModel_pred,testevaltarger))
print('Precision Score : ',precision_score(Test_RandomModel_pred,testevaltarger))

Accuracy Score :  0.7277027027027027
F1 Score :  0.735042735042735
Recall Score :  0.8469696969696969
Precision Score :  0.6492450638792102
